In [1]:
from shared.simulators import ConstantEffectSimulator, ModeratedEffectSimulator, DataSimulator, EffectsSimulatorCollection
from shared.analyzers import AnalyzerCollection, WLSAnalyzer, IPWAnalyzer
from shared.result_processors import Results
from tqdm import trange
import matplotlib.pyplot as plt

In [2]:
N = 50
T = 100
alpha = 0.05
n_reps = 100
weight_type = "ATE"

# Effect simulators
effect_simulators = EffectsSimulatorCollection()
effect_simulators.add_effect_simulator(ConstantEffectSimulator("No Effect", N, 0., 0., 0.))
effect_simulators.add_effect_simulator(ConstantEffectSimulator("Shared Effect", N, 1., 1., 0.))
effect_simulators.add_effect_simulator(ConstantEffectSimulator("Distinct Effects", N, 1., 1., 1.))
effect_simulators.add_effect_simulator(ModeratedEffectSimulator("Moderated Effects", N, 1., 1., 1., 1., 0.5))
effect_simulators.add_effect_simulator(ModeratedEffectSimulator("Noisy Moderated Effects", N, 1., 1., 1., 1., 0.5, epsilon_sd=1.))

# Simulate effects once and fix
for effect_simulator in effect_simulators:
    effect_simulator.sample_params()

# Analyzers
analyzers = AnalyzerCollection()
analyzers.add_analyzer(WLSAnalyzer("WLS Analyzer", alpha))
analyzers.add_analyzer(IPWAnalyzer("IPW Analyzer", alpha))

# Simulation loop
data_simulator = DataSimulator(N, T).simulate_history()
results = Results(data_simulator, effect_simulators, analyzers, n_reps=n_reps)
for rep_idx in trange(n_reps):
    for effect_simulator in effect_simulators:
        data_simulator = data_simulator.simulate_a()
        data_simulator = data_simulator.simulate_effects(effect_simulator)
        for analyzer in analyzers:          
            # Inferences for theta
            theta_inferences = analyzer.get_theta_inferences(data_simulator)
            results = results.add_theta_inferences(theta_inferences, data_simulator.theta.copy(), rep_idx, effect_simulator.i, analyzer.i)
            
            # Inferences for user estimates
            user_inferences = analyzer.get_user_inferences(data_simulator)
            results = results.add_user_inferences(user_inferences, data_simulator.user_effects.copy(), rep_idx, effect_simulator.i, analyzer.i)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 120.95it/s]


In [6]:
digits = 3
results = results.process(digits=digits)
results.user_results_df

WLS Analyzer   IPW Analyzer
Coefficient Metric       Effect Simulator                                     
Intercept   Avg Bias     No Effect                        0.168          0.007
                         Shared Effect                    0.161          0.002
                         Distinct Effects                 0.163          0.007
                         Moderated Effects                0.446          0.004
                         Noisy Moderated Effects          0.427         -0.003
            Avg Sq. Bias No Effect                        0.034          0.001
                         Shared Effect                    0.032          0.001
                         Distinct Effects                 0.033          0.001
                         Moderated Effects                0.229          0.002
                         Noisy Moderated Effects          0.220          0.001
            MSE          No Effect                0.079 (0.001)  0.055 (0.001)
                         Shared Effect            0.075 (0.002)  0.071 (0.002)
                         Distinct Effects         0.078 (0.001)  0.126 (0.003)
                         Moderated Effects        0.298 (0.004)  0.155 (0.003)
                         Noisy Moderated Effects  0.298 (0.005)  0.121 (0.002)
            Coverage     No Effect                0.877 (0.005)  0.948 (0.003)
                         Shared Effect            0.879 (0.005)  0.974 (0.002)
                         Distinct Effects         0.875 (0.004)  0.980 (0.002)
                         Moderated Effects        0.668 (0.006)  0.986 (0.002)
                         Noisy Moderated Effects  0.754 (0.006)  0.982 (0.002)
            CI Length    No Effect                0.855 (0.000)  0.910 (0.001)
                         Shared Effect            0.856 (0.000)  1.209 (0.001)
                         Distinct Effects         0.855 (0.000)  1.664 (0.001)
                         Moderated Effects        1.126 (0.001)  1.996 (0.001)
                         Noisy Moderated Effects  1.251 (0.001)  1.669 (0.002)